In [ ]:
!pip install tensorflow keras
!pip install pillow
!pip install tensorflow-model-optimization tf-keras
!pip install scikit-learn

In [ ]:
import tensorflow as tf
from keras import regularizers
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, Adamax



In [ ]:
# Define paths
dataset_dir = 'dataset_ab'
train_dir = dataset_dir 
val_dir = dataset_dir  # You can split data into train/val if needed

# Image parameters
img_height, img_width = 96, 96  # Adjust as needed
batch_size = 32

def to_grayscale(image):
    # image = tf.image.rgb_to_grayscale(image)
    return image

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=to_grayscale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=to_grayscale
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Use 'binary' for 2 classes
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Use 'binary' for 2 classes
)

In [ ]:

# Load pre-trained MobileNetV3Small
base_model = MobileNetV3Small(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = True

x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(32, kernel_regularizer = regularizers.l2(l2 = 0.016),activity_regularizer=regularizers.l1(l1 = 0.006),
                bias_regularizer=regularizers.l1(l1 = 0.006) ,activation='relu')(x)
x=Dropout(rate=.4, seed=69)(x)       

output=Dense(2, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy']) 

model.fit(
    train_generator,
    epochs=epochs,  # Train for a few more epochs
    validation_data=val_generator
)



In [ ]:
from tensorflow.keras import saving

# Save the model
try:
    #model.save("model_outputs/mobilenetv3_simple_ab.keras")    
except:
    @saving.register_keras_serializable()
    def F1_score(y_true, y_pred): #taken from old keras source code
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        recall = true_positives / (possible_positives + K.epsilon())
        f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
        return f1_val
    
    model = tf.keras.models.load_model('model_outputs/mobilenetv3_simple_ab.keras')



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np

# Define a representative dataset generator
def representative_dataset():
    dataset_list = tf.data.Dataset.list_files(dataset_dir + '/*/*')
    for i in range(2000):
        image_path = next(iter(dataset_list)).numpy().decode("utf-8")
        img = load_img(image_path, target_size=(img_height, img_width))
        img = img_to_array(img)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)
        yield [img.astype(np.float32)]
        
# Create the TFLite converter with INT8 quantization and representative dataset
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_dataset
tflite_model = converter.convert()   

# Save the TFLite model
with open('model_outputs/mobilenetv3_simple_ab.tflite', 'wb') as f:
    f.write(tflite_model)

In [52]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np

# Testing block
def test_image(image_path):
    """
    Loads an image, preprocesses it, and predicts its class using the trained model.

    Args:
        image_path (str): Path to the image file.
    """
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    class_labels = list(train_generator.class_indices.keys())  # Get class labels
    confidence = np.max(prediction)  # Get the highest probability

    print(prediction);
    print(class_labels);
    print("Predicted class: %s (confidence: %.2f)" % (class_labels[class_index], confidence))


# Example usage
test_image('test_inputs/not_cat2.jpg') 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
[[0.04815346 0.9518465 ]]
['cat', 'not_cat']
Predicted class: not_cat (confidence: 0.95)


In [53]:
from tensorflow.lite.python.util import convert_bytes_to_c_source

file_name = 'model'
source_text, header_text = convert_bytes_to_c_source(tflite_model,  file_name)

with  open('model_outputs/' + file_name + '.h',  'w')  as  file:
    file.write(header_text)

with  open('model_outputs/' + file_name + '.cc',  'w')  as  file:
    file.write(source_text)
